# Day 20
## Part 1

The biggest problem with this is going to be parsing the input. Turn it into a big graph with a node at each coordinate and an edge to each neighbour. Look for portals, track them, and add edges to the graph. Then do a breadth first search from AA to ZZ.

In [30]:
from collections import defaultdict, deque
import itertools


def parse_data(s):
    graph = defaultdict(list)
    portals = defaultdict(list)
    maze = s.splitlines()
    
    for row, line in enumerate(maze):
        for col, c in enumerate(line):
            if c == '.':
                for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    nbr = maze[row + dr][col + dc]
                    if nbr == '.':
                        graph[(row, col)].append((row + dr, col + dc))
                    elif nbr.isalpha():
                        # Get the next letter of the portal
                        portal = nbr + maze[row + 2 * dr][col + 2 * dc]
                        # Reverse if going north or west
                        if dr == -1 or dc == -1:
                            portal = ''.join(reversed(portal))
                        portals[portal].append((row, col))

    for portal in portals:
        if len(portals[portal]) > 1:
            for coord1, coord2 in itertools.permutations(portals[portal], 2):
                graph[coord1].append(coord2)
                
    return (graph, portals['AA'][0], portals['ZZ'][0])

In [31]:
test_maze_1 = ('''
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       ''')

parse_data(test_maze_1)

(defaultdict(list,
             {(3, 9): [(4, 9)],
              (4, 9): [(5, 9), (3, 9), (4, 10)],
              (4, 10): [(4, 11), (4, 9)],
              (4, 11): [(4, 12), (4, 10)],
              (4, 12): [(4, 13), (4, 11)],
              (4, 13): [(4, 14), (4, 12)],
              (4, 14): [(4, 15), (4, 13)],
              (4, 15): [(4, 16), (4, 14)],
              (4, 16): [(4, 17), (4, 15)],
              (4, 17): [(5, 17), (4, 16)],
              (5, 9): [(6, 9), (4, 9)],
              (5, 17): [(6, 17), (4, 17)],
              (6, 9): [(7, 9), (5, 9)],
              (6, 17): [(7, 17), (5, 17)],
              (7, 9): [(6, 9), (9, 2)],
              (7, 17): [(8, 17), (6, 17)],
              (8, 17): [(9, 17), (7, 17)],
              (9, 2): [(9, 3), (7, 9)],
              (9, 3): [(9, 4), (9, 2)],
              (9, 4): [(10, 4), (9, 3)],
              (9, 17): [(10, 17), (8, 17)],
              (10, 4): [(11, 4), (9, 4)],
              (10, 17): [(11, 17), (9, 17)],
             

In [40]:
def part_1(s):
    graph, start, end = parse_data(s)
    search = deque([(start, 0)])
    seen = {start}
    
    while search:
        node, steps_taken = search.popleft()
        for nbr in graph[node]:
            if nbr not in seen:
                if nbr == end:
                    return steps_taken + 1
                else:
                    seen.add(nbr)
                    search.append((nbr, steps_taken + 1))

In [41]:
%time assert part_1(test_maze_1) == 23

CPU times: user 329 µs, sys: 20 µs, total: 349 µs
Wall time: 371 µs


~That's a bit slow but never mind.~ I wasn't testing to see if the node had been seen before. 

In [42]:
test_maze_2 = '''

                   A               
                   A               
  #################.#############  
  #.#...#...................#.#.#  
  #.#.#.###.###.###.#########.#.#  
  #.#.#.......#...#.....#.#.#...#  
  #.#########.###.#####.#.#.###.#  
  #.............#.#.....#.......#  
  ###.###########.###.#####.#.#.#  
  #.....#        A   C    #.#.#.#  
  #######        S   P    #####.#  
  #.#...#                 #......VT
  #.#.#.#                 #.#####  
  #...#.#               YN....#.#  
  #.###.#                 #####.#  
DI....#.#                 #.....#  
  #####.#                 #.###.#  
ZZ......#               QG....#..AS
  ###.###                 #######  
JO..#.#.#                 #.....#  
  #.#.#.#                 ###.#.#  
  #...#..DI             BU....#..LF
  #####.#                 #.#####  
YN......#               VT..#....QG
  #.###.#                 #.###.#  
  #.#...#                 #.....#  
  ###.###    J L     J    #.#.###  
  #.....#    O F     P    #.#...#  
  #.###.#####.#.#####.#####.###.#  
  #...#.#.#...#.....#.....#.#...#  
  #.#####.###.###.#.#.#########.#  
  #...#.#.....#...#.#.#.#.....#.#  
  #.###.#####.###.###.#.#.#######  
  #.#.........#...#.............#  
  #########.###.###.#############  
           B   J   C               
           U   P   P               
'''

assert part_1(test_maze_2) == 58

In [49]:
maze = open('input').read()

In [50]:
part_1(maze)

608

## Part 2

I'm going to have to reparse the data to detect the outer wall. Let's just put it all into one big function, there's too much to keep track of to pass as return values and parameters. I should create a data structure really.

In [69]:
def part_2(s):
    graph = defaultdict(list)
    portals = defaultdict(list)
    maze = s.splitlines()
    
    wall_coords = [
        (row, col) 
        for row, line in enumerate(maze)
        for col, c in enumerate(line)
        if c == '#'
    ]
    row_bounds = (min(row for row, _ in wall_coords), 
                  max(row for row, _ in wall_coords))
    col_bounds = (min(col for _, col in wall_coords), 
                  max(col for _, col in wall_coords))
    
    for row, line in enumerate(maze):
        for col, c in enumerate(line):
            if c == '.':
                for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                    nbr = maze[row + dr][col + dc]
                    if nbr == '.':
                        # Each edge is now associated with a neighbour's position
                        # and a change in the maze level, which is zero when
                        # moving from dot to dot.
                        graph[(row, col)].append((row + dr, col + dc, 0))
                    elif nbr.isalpha():
                        # Get the next letter of the portal
                        portal = nbr + maze[row + 2 * dr][col + 2 * dc]
                        # Reverse if going north or west
                        if dr == -1 or dc == -1:
                            portal = ''.join(reversed(portal))
                        portals[portal].append((row, col))

    labels = defaultdict(str)
    for portal in portals:
        if len(portals[portal]) > 1:
            for coord1, coord2 in itertools.permutations(portals[portal], 2):
                r1, c1 = coord1
                r2, c2 = coord2
                # If a portal is on the outer wall go to the outermost level
                if r1 in row_bounds or c1 in col_bounds:
                    graph[(r1, c1)].append((r2, c2, -1))
                # otherwise go in a level
                else:
                    graph[(r1, c1)].append((r2, c2, 1))
        for coord in portals[portal]:
            labels[coord] = portal
            
    start = portals['AA'][0]
    start_r, start_c = start
    end = portals['ZZ'][0]
    search = deque([((start_r, start_c, 0), 0, 1)])
    seen = {start}
    
    while search:
        node, steps_taken, level = search.popleft()
        r, c, _ = node
        for nbr in graph[(r, c)]:
            row, col, dlevel = nbr
            if (row, col, level + dlevel) not in seen:
                if (row, col) == end and level == 1:
                    return steps_taken + 1
                elif (next_level := level + dlevel) > 0:
                    seen.add((row, col, next_level))
                    search.append((nbr, steps_taken + 1, next_level))                

In [70]:
assert part_2(test_maze_1) == 26

In [71]:
test_maze_3 = '''             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M                     '''

assert part_2(test_maze_3) == 396

In [72]:
part_2(maze)

6706

Good grief, that horrendous mess actually worked.